# Exercise 2 : Implementing Backpropagation with Python

In [ ]:
import numpy as np

In [ ]:
class NeuralNetwork:
    def __init__(self, layers, alpha=0.1):

        self.W = []
        self.layers = layers
        self.alpha = alpha

        # start looping from the index of the first layer but
        # stop before we reach the last two layers
        for i in np.arange(0, len(layers) - 2):
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.W.append(w / np.sqrt(layers[i]))

        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.W.append(w / np.sqrt(layers[-2]))

    def __repr__(self):
        return "NeuralNetwork: {}".format(
            "-".join(str(l) for l in self.layers))

    def sigmoid(self, x):
        return 1.0 / (1 + np.exp(-x))

    def sigmoid_deriv(self, x):
        return x * (1 - x)

    def fit(self, X, y, epochs=1000, displayUpdate=100):
        X = np.c_[X, np.ones((X.shape[0]))]

        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                self.fit_partial(x, target)

            if epoch == 0 or (epoch + 1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] epoch={}, loss={:.7f}".format(
                    epoch + 1, loss))

    def fit_partial(self, x, y):
        A = [np.atleast_2d(x)]

        # FEEDFORWARD:
        # loop over the layers in the network
        for layer in np.arange(0, len(self.W)):
            net = A[layer].dot(self.W[layer])

            out = self.sigmoid(net)

            A.append(out)

        # BACKPROPAGATION
        error = A[-1] - y

        D = [error * self.sigmoid_deriv(A[-1])]

        for layer in np.arange(len(A) - 2, 0, -1):
            delta = D[-1].dot(self.W[layer].T)
            delta = delta * self.sigmoid_deriv(A[layer])
            D.append(delta)

        D = D[::-1]

        # WEIGHT UPDATE PHASE
        # loop over the layers
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    def predict(self, X, addBias=True):
        p = np.atleast_2d(X)

        # check to see if the bias column should be added
        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]

        # loop over our layers in the network
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))

        return p

    def calculate_loss(self, X, targets):
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)

        return loss

### Neural Network XOR

In [ ]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

In [ ]:
nn = NeuralNetwork([2, 2, 1], alpha=0.5)    # xy값, 노드두개(노드한개면 퍼셉트론), output

In [ ]:
print(nn)

NeuralNetwork: 2-2-1


In [ ]:
nn.fit(X, y, epochs=20000, displayUpdate=2000)

[INFO] epoch=1, loss=0.5174323
[INFO] epoch=2000, loss=0.0047366
[INFO] epoch=4000, loss=0.0014448
[INFO] epoch=6000, loss=0.0008290
[INFO] epoch=8000, loss=0.0005761
[INFO] epoch=10000, loss=0.0004396
[INFO] epoch=12000, loss=0.0003545
[INFO] epoch=14000, loss=0.0002966
[INFO] epoch=16000, loss=0.0002546
[INFO] epoch=18000, loss=0.0002229
[INFO] epoch=20000, loss=0.0001981


In [ ]:
for (x, target) in zip(X, y):
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
                        x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.0087, step=0
[INFO] data=[0 1], ground-truth=1, pred=0.9902, step=1
[INFO] data=[1 0], ground-truth=1, pred=0.9902, step=1
[INFO] data=[1 1], ground-truth=0, pred=0.0113, step=0


### Neural Network MNIST

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import datasets

pixel intensity values to the range [0, 1]

each image is represented by an 8 x 8 = 64-dim feature vector

In [ ]:
digits = datasets.load_digits()

In [ ]:
digits.images[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(digits.images[100])

In [ ]:
digits.target[100]

construct the training and testing splits

In [ ]:
data = digits.data.astype("float")
data = (data - data.min()) / (data.max() - data.min())
print("[INFO] samples: {}, dim: {}".format(data.shape[0], data.shape[1]))

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data,
    digits.target, test_size=0.25)

In [ ]:
trainY[0]

convert the labels from integers to vectors

In [ ]:
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [ ]:
trainY[0]

train the network

In [ ]:
nn = NeuralNetwork([trainX.shape[1], 32, 16, 10])
print("[INFO] {}".format(nn))
nn.fit(trainX, trainY, epochs=1000)

evaluate the network

In [ ]:
predictions = nn.predict(testX)
predictions = predictions.argmax(axis=1)
print(classification_report(testY.argmax(axis=1), predictions))